Classification of the bacteria data using three different classifiers:
1. SVM
2. RandomForest with decision stumps
3. AdaBoost with decision stumps.

The data is each replicate of each species along the rows, and the fluorescense spectra readings at the stat growth phase along the columns. 

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/mnt/d/dev/summer-bursary-2018/bacteria'

In [3]:
os.listdir('data')

['16ms_32ms_growth_phase_spectra.csv',
 '16_ms_lag_codes.csv',
 'bacteria.csv',
 'Classific.py',
 'graphs.pdf',
 'wavelengths.csv']

In [4]:
df = pd.read_csv('data/bacteria.csv', header=[0, 1, 2], index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Columns: 135 entries, (lag, bc, 01) to (stat, se, 07)
dtypes: float64(135)
memory usage: 1.1 MB


# Preprocessing

In [5]:
stat = df['stat'].T
stat

0         1         2          3             4     \
species replicate                                                          
bc      01        -0.204750 -2.934770 -0.204750   0.614292  2.047664e+00   
        02         0.383045 -2.170600 -0.383050  -2.106860  3.192220e-01   
        03        -0.279776  0.419667 -0.419668   0.279793  2.378291e+00   
        04        -0.691798  0.314453  0.754683   0.125783  1.257884e+00   
        05        -0.537370 -0.000003 -0.134340   1.074790  1.343486e+00   
        06         0.361383 -0.542070 -0.782990   0.361400  1.325136e+00   
        07         0.307881 -0.800487 -0.184728   3.263684  9.236899e-01   
        08         1.835246  0.696122 -2.341510   0.696138 -1.265700e-01   
        09         1.202269 -1.075700 -2.088140  -1.075760  1.645302e+00   
        10        -4.740550  1.309730  0.311845 -11.824700 -1.019990e+01   
        11         0.127427 -0.318560  0.127421   1.401728 -6.371600e-01   
        12         1.137996 -0.695440 -0.252890   1.327726  8.219370e-01   
ec      01        -0.063290 -2.088610  0.063286  -1.708970  1.962151e+00   
        02         0.193644 -2.130098 -0.064553  -0.774617 -3.873147e-01   
        03        -1.979568 -0.219950  0.659852   0.513255  2.126323e+00   
        04        -1.664750  1.294805 -0.184980  -1.664850  1.171563e+00   
        05        -0.734830 -1.028750 -0.734820  -0.146970  1.616708e+00   
        06         0.130638 -0.653180 -2.155490  -0.130640 -3.919300e-01   
        07         3.839217  0.715669  0.195209  -1.761620 -5.090900e+00   
        08        -1.082790  0.473714 -0.135340  -0.541420  1.150526e+00   
        09         0.188591 -0.942950 -1.257260   2.074584  1.257332e+00   
lm      01        -1.861190 -4.141850  7.023941   0.180280  4.398416e+00   
        02        -0.580300  0.257906  0.257918  -0.580330 -5.803300e-01   
        03        -0.570440 -0.253520 -0.126770   3.929872  2.091718e+00   
pa      01         1.167576 -1.659180 -1.229020  -2.704000 -4.310000e-07   
        02         1.165914  0.632922  0.632917   0.899468 -1.232613e+00   
        03        -1.220729 -1.156476  1.027980   2.505856  2.377369e+00   
        04        -0.501250 -1.754370  0.438594  -1.002560 -2.945050e+00   
        05         1.235617 -0.845420 -1.690850  -0.780430  1.495836e+00   
        06        -0.060820 -0.547440  1.581496   2.372369  5.474790e-01   
        07         0.192142 -0.384280  1.024756  -0.000003  1.024821e+00   
        08         1.786078  0.184770 -0.923820  -3.264350 -1.539810e+00   
sa      01         0.480070  0.548650 -1.645950  -1.646040 -1.165960e+00   
        02         1.229529  0.361627  0.650924   0.650965 -1.084943e+00   
        03        -1.134910 -1.639300  1.008802   2.900473  2.585225e+00   
        04         0.259780 -2.013260  1.298883   1.298952  1.104111e+00   
        05         0.433044  1.422837  0.866080   0.433058 -6.187000e-02   
        06         1.757536 -2.071090 -2.071340   3.714417 -2.644110e+00   
        07        -0.525800 -0.328620 -0.985860   1.117374  5.915570e-01   
        08        -0.182860 -0.853340 -0.182860   2.255372  9.753000e-01   
se      01         0.918231 -0.655870  0.393521  -1.180630  1.246246e+00   
        02         1.283523  0.192528  1.219347  -3.337340  1.604498e+00   
        03        -0.734724 -1.269057 -1.669815  -0.667963  8.683631e-01   
        04         0.183516 -1.345770  0.367029   1.590540  9.176300e-01   
        05         2.187136 -1.004900 -3.369360  -1.477880 -8.867300e-01   
        06        -0.774600 -0.193650 -1.032800  -0.645540  1.420193e+00   
        07         0.423758  1.089653 -0.363220  -1.574020 -1.816200e+00   

                       5          6         7          8          9     \
species replicate                                                        
bc      01         1.433361   3.071483  0.614301  -0.750800   1.979407   
        02        -0.510760  -0.255380 -1.276900   0.127691  -0.063840   
  

In [6]:
stat.isnull().values.any()

False

In [7]:
X = stat

In [8]:
y = X.reset_index()['species']
y.head()

0    bc
1    bc
2    bc
3    bc
4    bc
Name: species, dtype: object

In [9]:
y.describe()

count     47
unique     6
top       bc
freq      12
Name: species, dtype: object

In [10]:
print(y)

0     bc
1     bc
2     bc
3     bc
4     bc
5     bc
6     bc
7     bc
8     bc
9     bc
10    bc
11    bc
12    ec
13    ec
14    ec
15    ec
16    ec
17    ec
18    ec
19    ec
20    ec
21    lm
22    lm
23    lm
24    pa
25    pa
26    pa
27    pa
28    pa
29    pa
30    pa
31    pa
32    sa
33    sa
34    sa
35    sa
36    sa
37    sa
38    sa
39    sa
40    se
41    se
42    se
43    se
44    se
45    se
46    se
Name: species, dtype: object


In [11]:
from sklearn.utils import shuffle

X, y = shuffle(X, y)

print(X[:5])
print(y[:5])

                       0         1         2         3         4         5     \
species replicate                                                               
ec      04        -1.664750  1.294805 -0.184980 -1.664850  1.171563  0.308304   
se      06        -0.774600 -0.193650 -1.032800 -0.645540  1.420193  1.742950   
bc      01        -0.204750 -2.934770 -0.204750  0.614292  2.047664  1.433361   
sa      05         0.433044  1.422837  0.866080  0.433058 -0.061870  3.588245   
ec      03        -1.979568 -0.219950  0.659852  0.513255  2.126323  0.366614   

                       6         7         8         9       ...         1033  \
species replicate                                            ...                
ec      04         0.308306  3.021410  1.048245  1.048222    ...     2.651889   
se      06         0.839207 -1.355650  0.387321 -0.839190    ...     4.648454   
bc      01         3.071483  0.614301 -0.750800  1.979407    ...     6.552984   
sa      05         1.608529

# PCA + SVM

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99, svd_solver='full')
pca.fit(X)

X_pca = pca.transform(X)
print(X_pca.shape)

(47, 1)


In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, cross_val_score, GridSearchCV

param_grid = {
    'kernel': ['linear', 'rbf'],
    'gamma': [10 ** -n for n in range(6)],
    'C': [10 ** n for n in range(-5, 2)]
}

cv = StratifiedKFold(n_splits=3)
clf = SVC()

grid_search = GridSearchCV(clf, param_grid, cv=cv, verbose=10, n_jobs=4)
grid_search.fit(X_pca, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 3 folds for each of 84 candidates, totalling 252 fits
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV]  C=1e-05, gamma=1, kernel=linear, score=0.5294117647058824, total=   0.0s
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV] .... C=1e-05, gamma=1, kernel=linear, score=0.5625, total=   0.0s
[CV]  C=1e-05, gamma=1, kernel=linear, score=0.5714285714285714, total=   0.0s
[CV] ......... C=1e-05, gamma=1, kernel=rbf, score=0.25, total=   0.0s
[CV]  C=1e-05, gamma=1, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV] C=1e-05, gamma=0.1, kernel=linear ...............................
[CV]  C=1e-05, gamma=1, kernel=rbf, score=0.2857142857142857, to

[Parallel(n_jobs=4)]: Batch computation too fast (0.0242s.) Setting batch_size=16.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1739s.) Setting batch_size=36.


[CV]  C=0.0001, gamma=0.1, kernel=linear, score=0.5294117647058824, total=   3.8s
[CV] C=0.0001, gamma=0.1, kernel=linear ..............................
[CV]  C=0.0001, gamma=1, kernel=linear, score=0.5294117647058824, total=   3.9s
[CV] C=0.0001, gamma=1, kernel=linear ................................
[CV]  C=0.0001, gamma=0.001, kernel=linear, score=0.5714285714285714, total=   4.2s
[CV] C=0.0001, gamma=0.001, kernel=rbf ...............................
[CV]  C=0.0001, gamma=0.001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.0001, gamma=0.001, kernel=rbf ...............................
[CV] .... C=0.0001, gamma=0.001, kernel=rbf, score=0.25, total=   0.0s
[CV] C=0.0001, gamma=0.001, kernel=rbf ...............................
[CV]  C=0.0001, gamma=0.001, kernel=rbf, score=0.2857142857142857, total=   0.0s
[CV] C=0.0001, gamma=0.0001, kernel=linear ...........................
[CV]  C=0.001, gamma=1, kernel=linear, score=0.5294117647058824, total=   6.2s
[CV] C=0.001, g

[Parallel(n_jobs=4)]: Done  40 tasks      | elapsed:   11.8s


[CV] .... C=0.001, gamma=1, kernel=linear, score=0.5625, total=   8.2s
[CV] C=0.001, gamma=1, kernel=linear .................................
[CV]  C=0.0001, gamma=0.01, kernel=linear, score=0.5294117647058824, total=   3.6s
[CV] C=0.0001, gamma=0.01, kernel=linear .............................
[CV]  C=0.0001, gamma=0.0001, kernel=linear, score=0.5714285714285714, total=   4.1s
[CV] C=0.0001, gamma=0.0001, kernel=rbf ..............................
[CV]  C=0.0001, gamma=0.0001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.0001, gamma=0.0001, kernel=rbf ..............................
[CV] ... C=0.0001, gamma=0.0001, kernel=rbf, score=0.25, total=   0.0s
[CV] C=0.0001, gamma=0.0001, kernel=rbf ..............................
[CV]  C=0.0001, gamma=0.0001, kernel=rbf, score=0.2857142857142857, total=   0.0s
[CV] C=0.0001, gamma=1e-05, kernel=linear ............................
[CV]  C=0.001, gamma=0.001, kernel=linear, score=0.5294117647058824, total=   5.9s
[CV] C=0.001, ga

[CV] C=0.01, gamma=1e-05, kernel=linear ..............................
[CV] ..... C=0.01, gamma=0.01, kernel=linear, score=0.5, total=  14.3s
[CV] C=0.01, gamma=0.01, kernel=rbf ..................................
[CV]  C=0.01, gamma=0.01, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.01, gamma=0.01, kernel=rbf ..................................
[CV] ....... C=0.01, gamma=0.01, kernel=rbf, score=0.25, total=   0.0s
[CV] C=0.01, gamma=0.01, kernel=rbf ..................................
[CV]  C=0.01, gamma=0.01, kernel=rbf, score=0.2857142857142857, total=   0.0s
[CV] C=0.01, gamma=0.001, kernel=linear ..............................
[CV]  C=0.001, gamma=1e-05, kernel=linear, score=0.5625, total=   8.4s
[CV] C=0.1, gamma=0.0001, kernel=rbf .................................
[CV] ...... C=0.1, gamma=0.0001, kernel=rbf, score=0.25, total=   0.0s
[CV] C=0.1, gamma=0.0001, kernel=rbf .................................
[CV]  C=0.1, gamma=0.0001, kernel=rbf, score=0.285714285714285

[CV] ......... C=1, gamma=1e-05, kernel=rbf, score=0.25, total=   0.0s
[CV] C=1, gamma=1e-05, kernel=rbf ....................................
[CV]  C=1, gamma=1e-05, kernel=rbf, score=0.35714285714285715, total=   0.0s
[CV] C=10, gamma=1, kernel=linear ....................................
[CV] ........ C=1, gamma=1, kernel=linear, score=0.6875, total= 1.4min
[CV] C=1, gamma=1, kernel=linear .....................................
[CV]  C=10, gamma=1e-05, kernel=linear, score=0.5294117647058824, total= 3.7min
[CV] C=10, gamma=1e-05, kernel=linear ................................
[CV]  C=0.1, gamma=0.1, kernel=linear, score=0.5714285714285714, total=  57.3s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV]  C=0.1, gamma=0.1, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ......... C=0.1, gamma=0.1, kernel=rbf, score=0.25, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ..............

[CV]  C=1, gamma=0.001, kernel=rbf, score=0.2857142857142857, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] ..... C=10, gamma=0.1, kernel=linear, score=0.6875, total= 1.8min
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV]  C=1, gamma=0.0001, kernel=linear, score=0.5294117647058824, total=  58.6s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] ... C=1, gamma=0.0001, kernel=linear, score=0.6875, total= 1.2min
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV]  C=10, gamma=0.1, kernel=linear, score=0.5714285714285714, total= 1.6min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] .......... C=10, gamma=0.1, kernel=rbf, score=0.25, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf ..................

[Parallel(n_jobs=4)]: Done 252 out of 252 | elapsed: 35.1min finished


0.5957446808510638
{'C': 1, 'gamma': 1, 'kernel': 'linear'}


In [14]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.52 (+/- 0.18)
CPU times: user 36min 23s, sys: 78.1 ms, total: 36min 23s
Wall time: 36min 25s


In [15]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.54 (+/- 0.24)
CPU times: user 1.95 s, sys: 0 ns, total: 1.95 s
Wall time: 1.94 s


# Random Forest with Decision Stumps

In [16]:
%%time

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/home/anthony/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Accuracy: 0.55 (+/- 0.18)
CPU times: user 33.8 s, sys: 391 ms, total: 34.2 s
Wall time: 34.2 s


In [17]:
%%time

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.49 (+/- 0.15)
CPU times: user 36.4 s, sys: 156 ms, total: 36.5 s
Wall time: 36.5 s


# AdaBoost

In [18]:
%%time

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.38 (+/- 0.19)
CPU times: user 40.9 s, sys: 15.6 ms, total: 41 s
Wall time: 41 s


In [19]:
%%time

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.38 (+/- 0.15)
CPU times: user 2min 12s, sys: 46.9 ms, total: 2min 12s
Wall time: 2min 12s
